# Logistic Regression and SGD Homework 
***
**Name**: $<$insert name here$>$ 
***

This assignment is due on Moodle by **5pm on Friday February 9th**. Submit only this Jupyter notebook to Moodle.  Do not compress it using tar, rar, zip, etc. Your solutions to analysis questions should be done in Markdown directly below the associated question.  Remember that you are encouraged to discuss the problems with your instructors and classmates, but **you must write all code and solutions on your own**.  For a refresher on the course **Collaboration Policy** click [here](https://github.com/chrisketelsen/CSCI5622-Machine-Learning/blob/master/resources/syllabus.md#collaboration-policy)



## Overview 
***


In this homework you'll implement stochastic gradient ascent for logistic regression and you'll apply it to the task of determining whether documents are talking about automobiles or motorcycles.

<br>

![autos_motorcycles](autos_motorcycles.jpg "A car and a motorcycle")


<br>

You should not use any libraries that implement any of the functionality of logistic regression for this assignment; logistic regression is implemented in Scikit-Learn, but you should do everything by hand now. You'll be able to use library implementations of logistic regression in the future.

Here are the rules: 

- Do **NOT** load or use any Python packages that are not available in Anaconda 3.6. 
- Some problems with code may be autograded.  If we provide a function or class API **do not** change it.
- Do not change the location of the data or data directory.  Use only relative paths to access the data. 

In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline 

### [5 points] Problem 1: Loading and Exploring the Data
***

The `Example` class will be used to store the features and labels associated with a single training or test example.  The `read_data` function will read in the text data and split it into training and test sets.  

 Load the data and then do the following: 
- Report the number of words in the vocabulary 
- Explain how the code is creating features (i.e. what text model is being used). 
- Go into the raw text files in the data directory and figure out which label (0/1) refers to which class of document (automobiles or motorcycles)

In [8]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

import math
from collections import defaultdict

kSEED = 1735
kBIAS = "BIAS_CONSTANT"

np.random.seed(kSEED)

class Example:
    """
    Class to represent a document example
    """
    def __init__(self, label, words, vocab):
        """
        Create a new example

        :param label: The label (0 / 1) of the example
        :param words: The words in a list of "word:count" format
        :param vocab: The vocabulary to use as features (list)
        """
        self.nonzero = {}
        self.y = label
        self.x = np.zeros(len(vocab))
        for word, count in [x.split(":") for x in words]:
            if word in vocab:
                assert word != kBIAS, "Bias can't actually appear in document"
                self.x[vocab.index(word)] += float(count)
                self.nonzero[vocab.index(word)] = word
        self.x[0] = 1

def read_dataset(positive, negative, vocab, train_frac=0.9):
    """
    Reads in a text dataset with a given vocabulary

    :param positive: Positive examples
    :param negative: Negative examples
    :param vocab: A list of vocabulary words
    :param test_frac: How much of the data should be reserved for test
    """

    vocab = [x.split("\t")[0] for x in open(vocab, 'r') if '\t' in x]
    assert vocab[0] == kBIAS, \
        "First vocab word must be bias term (was %s)" % vocab[0]

    train_set = []
    test_set = []
    for label, input in [(1, positive), (0, negative)]:
        for line in open(input):
            ex = Example(label, line.split(), vocab)
            if np.random.random() <= train_frac:
                train_set.append(ex)
            else:
                test_set.append(ex)

    # Shuffle the data 
    np.random.shuffle(train_set)
    np.random.shuffle(test_set)

    return train_set, test_set, vocab


pos_fname = "positive"
neg_fname = "negative"
voc_fname = "vocab"
train_set, test_set, vocab = read_dataset(pos_fname, neg_fname, voc_fname)
print(len(vocab))

class LogReg:
    def __init__(self, train_set, test_set, lam, eta=0.1):
        """
        Create a logistic regression classifier

        :param train_set: A set of training examples
        :param test_set: A set of test examples 
        :param lam: Regularization parameter
        :param eta: The learning rate to use 
        """
        
        # Store training and test sets 
        self.train_set = train_set
        self.test_set = test_set 
        
        # Initialize vector of weights to zero  
        self.w = np.zeros_like(train_set[0].x)
        
        # Store regularization parameter and eta function 
        self.lam = lam
        self.eta = eta
        
        # Create dictionary for lazy-sparse regularization
        self.last_update = defaultdict(int)

        # Make sure regularization parameter is not negative 
        assert self.lam>= 0, "Regularization parameter must be non-negative"
        
        # Empty lists to store NLL and accuracy on train and test sets 
        self.train_nll = []
        self.test_nll = []
        self.train_acc = []
        self.test_acc = []
      
    def sigmoid(self,score, threshold=20.0):
        """
        Prevent overflow of exp by capping activation at 20.
        You do not need to change this function. 

        :param score: A real valued number to convert into a number between 0 and 1
        """

        # if score > threshold, cap value at score 
        if abs(score) > threshold:
            score = threshold * np.sign(score)

        return 1.0 / (1.0 + np.exp(-score)) 

    def compute_progress(self, examples):
        """
        Given a set of examples, compute the NLL and accuracy
        You shouldn't need to change this function. 

        :param examples: The dataset to score
        :return: A tuple of (log probability, accuracy)
        """

        NLL = 0.0
        num_correct = 0
        for ex in examples:
            # compute prob prediction
            p = self.sigmoid(self.w.dot(ex.x))
            # update negative log likelihood
            NLL = NLL - np.log(p) if ex.y==1 else NLL - np.log(1.0-p)
            # update number correct 
            num_correct += 1 if np.floor(p+.5)==ex.y else 0

        return NLL, float(num_correct) / float(len(examples))
    
    def train(self, num_epochs=1, isVerbose=False, report_step=5):
        """
        Train the logistic regression classifier on the training data 

        :param num_epochs: number of full passes over data to perform 
        :param isVerbose: boolean indicating whether to print progress
        :param report_step: how many iterations between recording progress
        """
        iteration = 0
        # Perform an epoch 
        for pp in range(num_epochs):
            # shuffle the data  
            np.random.shuffle(self.train_set)
            # loop over each training example
            for ex in self.train_set:
                # perform SGD update of weights
                iteration += 1
                self.sgd_update(ex, iteration)
                # record progress 
                if iteration % report_step == 1:
                    train_nll, train_acc = self.compute_progress(self.train_set)
                    test_nll, test_acc = self.compute_progress(self.test_set)
                    self.train_nll.append(train_nll)
                    self.test_nll.append(test_nll)
                    self.train_acc.append(train_acc)
                    self.test_acc.append(test_acc)
                    
                    if isVerbose:
                        print("Update {: 5d}  TrnNLL {: 8.3f}  TstNLL {: 8.3f}  TrnA {:.3f}  TstA {:.3f}"
                             .format(iteration, train_nll, test_nll, train_acc, test_acc))
                
    
    def sgd_update(self, train_example, iteration, use_tfidf=False):
        """
        Compute a stochastic gradient update to improve the log likelihood.
        :param train_example: The example to take the gradient with respect to
        :param iteration: The current iteration (an integer)
        :param use_tfidf: A boolean to switch between the raw data and the tfidf representation
        :return: Return the new value of the regression coefficients
        """
        
        idx = list(train_example.nonzero.keys())
        idx.append(0)
        
        x    =  train_example.x
        y    =  train_example.y

        ebiasx = math.exp(self.w[idx].dot(x[idx]))
        p = ebiasx/float(1+ebiasx)
        self.w[idx] = self.w[idx] + ((self.eta) * (y - p) * x[idx])
        biasunreg = self.w[0] 
        
        for i in range(0, len(self.w)):
            self.last_update[i] += 1
        regular = np.power((1 - (2 * self.eta * self.lam)),((list(self.last_update.values()))))
        self.w[idx] = self.w[idx] * regular[idx]
        
        self.w[0] = biasunreg


# %run -i tests.py "part A"
# %run -i tests.py "part B"
    
    def goodbbwords(self):
        return np.argsort(self.w)[-10:]
    def goodhwords(self):
        return np.argsort(self.w)[:10]
    def badwords(self):
        return np.argsort(abs(self.w))[:10] #Getting the zeros

def results(lr):
    print ("Motorcycle: ")
    baseball = lr.goodbbwords()
    print([vocab[i] for i in baseball])
    print("Car: ")
    hockey = lr.goodhwords()
    print([vocab[i] for i in hockey])
    print ("Bad: ")
    bad = lr.badwords()
    print([vocab[i] for i in bad])


lr = LogReg(train_set, test_set, lam=0.006, eta=0.1)
lr.train(isVerbose=True)
results(lr)

5327
Update     1  TrnNLL  733.385  TstNLL   77.689  TrnA 0.498  TstA 0.534
Update     6  TrnNLL  701.963  TstNLL   69.886  TrnA 0.613  TstA 0.612
Update    11  TrnNLL  3668.834  TstNLL  360.648  TrnA 0.508  TstA 0.552
Update    16  TrnNLL  3341.465  TstNLL  325.427  TrnA 0.519  TstA 0.552
Update    21  TrnNLL  2718.659  TstNLL  263.318  TrnA 0.530  TstA 0.578
Update    26  TrnNLL  2238.830  TstNLL  221.453  TrnA 0.565  TstA 0.595
Update    31  TrnNLL  1330.688  TstNLL  138.514  TrnA 0.664  TstA 0.655
Update    36  TrnNLL  1196.192  TstNLL  127.621  TrnA 0.675  TstA 0.681
Update    41  TrnNLL  531.950  TstNLL   73.362  TrnA 0.804  TstA 0.767
Update    46  TrnNLL  461.110  TstNLL   54.713  TrnA 0.833  TstA 0.810
Update    51  TrnNLL  444.985  TstNLL   52.886  TrnA 0.840  TstA 0.810
Update    56  TrnNLL  430.765  TstNLL   53.224  TrnA 0.837  TstA 0.802
Update    61  TrnNLL  417.186  TstNLL   50.156  TrnA 0.845  TstA 0.810
Update    66  TrnNLL  529.548  TstNLL   61.614  TrnA 0.814  TstA 0

Update   601  TrnNLL  444.619  TstNLL   51.830  TrnA 0.818  TstA 0.802
Update   606  TrnNLL  382.082  TstNLL   46.232  TrnA 0.943  TstA 0.905
Update   611  TrnNLL  399.922  TstNLL   49.190  TrnA 0.902  TstA 0.853
Update   616  TrnNLL  404.549  TstNLL   48.684  TrnA 0.929  TstA 0.914
Update   621  TrnNLL  409.620  TstNLL   49.329  TrnA 0.923  TstA 0.888
Update   626  TrnNLL  440.259  TstNLL   52.009  TrnA 0.832  TstA 0.853
Update   631  TrnNLL  410.997  TstNLL   49.097  TrnA 0.907  TstA 0.897
Update   636  TrnNLL  409.382  TstNLL   49.343  TrnA 0.937  TstA 0.922
Update   641  TrnNLL  418.111  TstNLL   51.288  TrnA 0.935  TstA 0.914
Update   646  TrnNLL  417.563  TstNLL   51.450  TrnA 0.929  TstA 0.905
Update   651  TrnNLL  450.265  TstNLL   55.909  TrnA 0.858  TstA 0.767
Update   656  TrnNLL  456.221  TstNLL   56.708  TrnA 0.858  TstA 0.784
Update   661  TrnNLL  432.423  TstNLL   53.961  TrnA 0.923  TstA 0.879
Update   666  TrnNLL  435.494  TstNLL   54.556  TrnA 0.940  TstA 0.888
Update

### [25 points] Problem 2: Implementing SGD with Lazy Sparse Regularization
***

We've given you a class `LogReg` below which will train a logistic regression classifier to predict whether a document is talking about automobiles or motorcycles. 

**Part A**: In this problem you will modify the `sgd_update` function to perform **unregularized** stochastic gradient descent updates of the weights. Note that you should only update the weights for **non-zero** features, i.e. weights associated with words that appear in the current training example. The code below this cell demonstrates how to instantiate the class and train the classifier.   

We've also given you unit tests in the next cell based on the simple example worked out in  the Lecture 4 in-class notebook.  At first your code will fail both of them. When your code is working you should pass tests called `test_unreg` and `test_learnrate`.  Do not move on to **Part A** until your code passes both of them. 

The unit tests are located in the script `tests.py` in this directory.  Execute the following cell to call the script and run the tests. 

**Part B**: After your unregularized updates are working, modify the `sgd_update` function again to perform regularized updates using **Lazy Sparse Regularization**. Note that you should not regularize the bias weight. See the Lecture 4 in-class notebook for a refresher on LSR. **Note**: After implementing LSR, your code should still pass the unit tests for **Part A** when `lam = 0`. 

We've given you a third unit test in the next cell called `test_reg` based on the simple example of LSR worked out in  the Lecture 4 in-class notebook.  Do not move on to **Problem 3** until your code passes the test. 

### [10 points] Problem 3: Hyperparameter Tuning 
***

**Part A**: Perform a systematic study of the effect of the regularization parameter on the accuracy of your classifier on the test set.  Which choice of `lam` seems to do the best?  Justify your conclusion with some kind of graphic. 

**Part B**: For the value of `lam` chosen in **Part A** perform a systematic study of the choice of learning rate on the speed of convergence SGD.  Which learning rate seems to give the fastest convergence?  Justify your conclusion with some kind of graphic. 

### [10 points] Problem 4: Identifying Predictive and Non-Predictive Words 
***

**Part A**: Find the top 10 words that are the best predictors for each class.  Explain mathematically how you identified them and show any code that you used to find them. 

**Part B**: Find the 10 words that are the worst predictors for class.  Explain mathematically how you identified them and show any code that you used to find them. 